In [1]:
import os
import pymysql
import json
import numpy as np
import pandas as pd

In [3]:
def connectdb():
    db = pymysql.connect(
        user = 'root',
        passwd = '110419',
        host = '127.0.0.1',
        db = 'knowledgebase',
        charset = 'utf8'
    )
    
    cursor = db.cursor(pymysql.cursors.DictCursor)
    
    return db, cursor

In [4]:
def preprocessing(path):
    with open(path + "/wmi.json", "r") as f:
        jsonString = json.load(f)
     
    proc_list = []
    
    for line in jsonString["WMIProcessList"]["SELECT * from Win32_Process"]:
        #print("Name: " + str(line["Caption"]))
        #print("Path: " + str(line["ExecutablePath"]))
        
        Name = str(line["Caption"])
        Path = str(line["ExecutablePath"])
        
        proc_list.append([str(Name), str(Path)])
        
    return proc_list

In [9]:
def extract_and_labeling():
    dir_list = os.listdir("is_before")
    dir_list2 = os.listdir("infected system")
    
    for infected in dir_list:
        df = pd.DataFrame(columns = ["Name", "Path", "Infected"])
        csv = pd.read_csv("infected system/" + infected + "/wmi.csv", index_col = 0)
        
        wmi = open("is_before/" + infected + "/wmi.json", "r")
        wmi_json = json.load(wmi)
        
        c = 0
        for obj in wmi_json["WMIProcessList"]["SELECT * from Win32_Process"]:
            Name = str(obj["Caption"])
            Path = str(obj["ExecutablePath"])
            Label = csv.loc[c]["f30"]
            df.loc[c] = [Name, Path, Label]
            c += 1
    
        df.to_csv("is_after/" + infected + "_wmi.csv")

In [9]:
path_nm = "normal01"
df_nm = pd.DataFrame(columns = ["Name", "Path", "Infected"])
wmi = open(path_nm + "/wmi.json", "r")
wmi_json = json.load(wmi)

c = 0
for obj in wmi_json["WMIProcessList"]["SELECT * from Win32_Process"]:
    Name = str(obj["Caption"])
    Path = str(obj["ExecutablePath"])
    Infected = str("0")
    df_nm.loc[c] = [Name, Path, Infected]
    c += 1

In [10]:
df_nm.to_csv("normal_for_e.csv")

In [156]:
extract_and_labeling()

In [13]:
dir_list = os.listdir("is_after")

for infected in dir_list:
    insert_data = []
    csv = pd.read_csv("is_after/" + infected, index_col = 0)
    cc += len(csv)
    for i in range(0, len(csv)):
        if csv.loc[i]["Infected"] == 0:
            insert_data.append([str(csv.loc[i]["Name"]), str(csv.loc[i]["Path"])])
            
    sql = 'insert into `proclist` values (%s, %s);'
    cursor.executemany(sql, insert_data)
    db.commit()

In [12]:
len(insert_data)

2282

In [18]:
print(['svchost.exe', 'C:\\WINDOWS\\System32\\svchost.exe'])
print(['svchost.exe', 'C:', 'WINDOWS', 'System32'])

['svchost.exe', 'C:\\WINDOWS\\System32\\svchost.exe']
['svchost.exe', 'C:', 'WINDOWS', 'System32']


In [15]:
with open("forkb/wmi.json", "r") as f:
    tmpString = json.load(f)

In [17]:
for line in tmpString["WMIProcessList"]["SELECT * from Win32_Process"]:
    print(json.dumps(line, indent = 2))

{
  "Caption": "System Idle Process",
  "CommandLine": null,
  "CreationClassName": "Win32_Process",
  "CreationDate": "20220413200942.286160+540",
  "CSCreationClassName": "Win32_ComputerSystem",
  "CSName": "DESKTOP-2RLGE9N",
  "Description": "System Idle Process",
  "ExecutablePath": null,
  "ExecutionState": null,
  "Handle": "0",
  "HandleCount": 0,
  "InstallDate": null,
  "KernelModeTime": "83853671250000",
  "MaximumWorkingSetSize": null,
  "MinimumWorkingSetSize": null,
  "Name": "System Idle Process",
  "OSCreationClassName": "Win32_OperatingSystem",
  "OSName": "Microsoft Windows 10 Home|C:\\WINDOWS|\\Device\\Harddisk0\\Partition3",
  "OtherOperationCount": "0",
  "OtherTransferCount": "0",
  "PageFaults": 9,
  "PageFileUsage": 60,
  "ParentProcessId": 0,
  "PeakPageFileUsage": 60,
  "PeakVirtualSize": "8192",
  "PeakWorkingSetSize": 12,
  "Priority": 0,
  "PrivatePageCount": "61440",
  "ProcessId": 0,
  "QuotaNonPagedPoolUsage": 1,
  "QuotaPagedPoolUsage": 0,
  "QuotaPeakNo

In [ ]:
with open("forkb/$MFT.json", "r") as f:
    jsonString = json.load(f)

for o in jsonString:
    path = o["FullPath"]
    name = path.split("\\")[-1]
    if path[-4:] == ".exe":
        print(name, "/", path)

In [43]:
dir_list = os.listdir("is_before")

for D in dir_list:
    df_mft = pd.DataFrame(columns = ["Name", "Path", "Infected"])
    
    with open("is_before/"+D+"/$MFT.json", "r") as f:
        mftjson = json.load(f)
    mftcsv = pd.read_csv("infected system/" + D + "/mft.csv", index_col = 0)
    
    c = 0
    for obj in mftjson:
        path = obj["FullPath"]
        name = path.split("\\")[-1]
        if path[-4:] == ".exe":
            df_mft.loc[c] = [name, path, mftcsv.loc[c]["Malicious"]]
            c += 1
    
    df_mft.to_csv("is_after/" + D + "_mft.csv")

In [48]:
db, cursor = connectdb()

In [49]:
dir_list = os.listdir("is_after")

for infected in dir_list:
    insert_data = []
    if infected[-7:-4] == "mft":
        csv = pd.read_csv("is_after/" + infected, index_col = 0)
        for i in range(0, len(csv)):
            if csv.loc[i]["Infected"] == 0:
                insert_data.append([str(csv.loc[i]["Name"]), str(csv.loc[i]["Path"])])
    
    sql = 'insert into `exelist` values (%s, %s);'
    cursor.executemany(sql, insert_data)
    db.commit()